In [1]:
# Import the data from the csv file
import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1117957 entries, 0 to 1117956
Data columns (total 22 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   id                               1117957 non-null  int64  
 1   MonsoonIntensity                 1117957 non-null  int64  
 2   TopographyDrainage               1117957 non-null  int64  
 3   RiverManagement                  1117957 non-null  int64  
 4   Deforestation                    1117957 non-null  int64  
 5   Urbanization                     1117957 non-null  int64  
 6   ClimateChange                    1117957 non-null  int64  
 7   DamsQuality                      1117957 non-null  int64  
 8   Siltation                        1117957 non-null  int64  
 9   AgriculturalPractices            1117957 non-null  int64  
 10  Encroachments                    1117957 non-null  int64  
 11  IneffectiveDisasterPreparedness  1117957 non-null 

In [3]:
# Importing Autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

# Load the data into AutoGluon
train_data = TabularDataset(train)
test_data = TabularDataset(test)

# Train the model
label = "FloodProbability"
metric = 'r2'  # specifies which metric to use

In [4]:
predictor = TabularPredictor(label=label, eval_metric=metric, problem_type="regression")

predictor.fit(train_data, time_limit=600, presets="best_quality", hyperparameters="multimodal")

No path specified. Models will be saved in: "AutogluonModels/ag-20240527_122131"
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240527_122131/ds_sub_fit/sub_fit_ho.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 165 seconds

In [5]:
predictor.leaderboard(train_data, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L2,0.861886,0.849377,r2,264.829142,122.128572,351.781819,24.534845,11.263553,80.255419,2,True,4
1,WeightedEnsemble_L3,0.861539,0.851022,r2,286.348881,132.780040,402.693856,0.016130,0.018706,1.275076,3,True,6
2,LightGBMXT_BAG_L2,0.857992,0.847444,r2,261.797906,121.497781,321.163361,21.503610,10.632761,49.636961,2,True,5
3,LightGBM_BAG_L1,0.849734,0.840533,r2,233.061303,107.478712,249.429587,233.061303,107.478712,249.429587,1,True,1
4,WeightedEnsemble_L2,0.849734,0.840533,r2,233.068677,107.495094,250.256340,0.007374,0.016382,0.826753,2,True,3
5,LightGBMXT_BAG_L1,0.365718,0.361183,r2,7.232993,3.386307,22.096812,7.232993,3.386307,22.096812,1,True,2


In [7]:
predictions = predictor.predict(test_data)

In [ ]:
op_sample = pd.read_csv('data/sample_submission.csv')
op_sample['FloodProbability'] = predictions
op_sample.to_csv('submission.csv', index=False)